# Image classification using TensorFlow

### Springboard Career Track Capstone Project 2

#### Linear Model in TensorFlow

#### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline 
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.pylabtools import figsize
figsize(12, 8)

#### Import data

Data acquired from https://www.cs.toronto.edu/~kriz/cifar.html

In [2]:
import tarfile
tar = tarfile.open("cifar-10-python.tar.gz")
tar.extractall()
tar.close()

#### Create function to unpickle data

In [3]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [4]:
def one_hot_encode(x):
    """
    One hot encode a list of sample labels. Return a one-hot encoded vector for each label.
    : x: List of sample Labels
    : return: Numpy array of one-hot encoded labels
    """  
    from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
    encoder = LabelBinarizer()
    encoder.fit(range(10))

    return encoder.transform(x)

#### Assign batch names, unpickle batches, and concatenate

In [5]:
f1 = 'cifar-10-batches-py\data_batch_1'
f2 = 'cifar-10-batches-py\data_batch_2'
f3 = 'cifar-10-batches-py\data_batch_3'
f4 = 'cifar-10-batches-py\data_batch_4'
f5 = 'cifar-10-batches-py\data_batch_5'
x_test = 'cifar-10-batches-py\\test_batch'
label_names = 'cifar-10-batches-py\\batches.meta'

In [6]:
b1 = unpickle(f1)

In [7]:
b1.keys()

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])

In [8]:
train_y1 = b1.get(b'labels')
b1data = b1.get(b'data')

b2 = unpickle(f2)
train_y2 = b2.get(b'labels')
b2data = b2.get(b'data')

b3 = unpickle(f3)
train_y3 = b3.get(b'labels')
b3data = b3.get(b'data')

b4 = unpickle(f4)
train_y4 = b4.get(b'labels')
b4data = b4.get(b'data')

b5 = unpickle(f5)
train_y5 = b5.get(b'labels')
b5data = b5.get(b'data')

x_train = np.concatenate((b1data, b2data, b3data, b4data, b5data), axis=0)
y_train = np.concatenate((train_y1, train_y2, train_y3, train_y4, train_y5), axis=0)

y_train_cls = y_train
y_train = one_hot_encode(y_train)

test = unpickle(x_test)
x_test = test.get(b'data')
y_test_cls = test.get(b'labels')
y_test = one_hot_encode(y_test_cls)
label_names = unpickle(label_names)

In [9]:
x_train.shape

(50000, 3072)

In [10]:
x_train

array([[ 59,  43,  50, ..., 140,  84,  72],
       [154, 126, 105, ..., 139, 142, 144],
       [255, 253, 253, ...,  83,  83,  84],
       ..., 
       [ 35,  40,  42, ...,  77,  66,  50],
       [189, 186, 185, ..., 169, 171, 171],
       [229, 236, 234, ..., 173, 162, 161]], dtype=uint8)

In [33]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ..., 
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [12]:
y_train.shape

(50000, 10)

In [13]:
label_names

{b'label_names': [b'airplane',
  b'automobile',
  b'bird',
  b'cat',
  b'deer',
  b'dog',
  b'frog',
  b'horse',
  b'ship',
  b'truck'],
 b'num_cases_per_batch': 10000,
 b'num_vis': 3072}

In [14]:
x_test.shape

(10000, 3072)

In [15]:
y_test.shape

(10000, 10)

In [16]:
# normalize inputs from 0-255 to 0-1
x_train = x_train / 255
x_test = x_test / 255

In [32]:
x_train

array([[ 0.23137255,  0.16862745,  0.19607843, ...,  0.54901961,
         0.32941176,  0.28235294],
       [ 0.60392157,  0.49411765,  0.41176471, ...,  0.54509804,
         0.55686275,  0.56470588],
       [ 1.        ,  0.99215686,  0.99215686, ...,  0.3254902 ,
         0.3254902 ,  0.32941176],
       ..., 
       [ 0.1372549 ,  0.15686275,  0.16470588, ...,  0.30196078,
         0.25882353,  0.19607843],
       [ 0.74117647,  0.72941176,  0.7254902 , ...,  0.6627451 ,
         0.67058824,  0.67058824],
       [ 0.89803922,  0.9254902 ,  0.91764706, ...,  0.67843137,
         0.63529412,  0.63137255]])

In [18]:
x_test

array([[ 0.61960784,  0.62352941,  0.64705882, ...,  0.48627451,
         0.50588235,  0.43137255],
       [ 0.92156863,  0.90588235,  0.90980392, ...,  0.69803922,
         0.74901961,  0.78039216],
       [ 0.61960784,  0.61960784,  0.54509804, ...,  0.03137255,
         0.01176471,  0.02745098],
       ..., 
       [ 0.07843137,  0.0745098 ,  0.05882353, ...,  0.19607843,
         0.20784314,  0.18431373],
       [ 0.09803922,  0.05882353,  0.09019608, ...,  0.31372549,
         0.31764706,  0.31372549],
       [ 0.28627451,  0.38431373,  0.38823529, ...,  0.36862745,
         0.22745098,  0.10196078]])

Using Keras (adapted from https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html)

Using baseline model Keras example from https://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-neural-networks-python-keras/

In [19]:
import tensorflow as tf
sess = tf.Session()

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import keras.utils

Using TensorFlow backend.


In [44]:
# define baseline model
def baseline_model(optimizer='SGD'):
    # create model
    np.random.seed(1337)
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
    
    # Compile model
    SGD = keras.optimizers.SGD(lr=learning_rate, momentum=0.0, decay=0.0, nesterov=False)
    RMSprop = keras.optimizers.RMSprop(lr=learning_rate, rho=0.9, epsilon=None, decay=0.0)
    Adagrad = keras.optimizers.Adagrad(lr=learning_rate, epsilon=None, decay=0.0)
    Adadelta = keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
    Adam = keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
    Adamax = keras.optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
    Nadam = keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

Fit original baseline model, no hyperparameter optimization

In [43]:
num_pixels = 32 * 32 * 3
num_classes = 10

from keras import optimizers

# build the model
model = baseline_model()
# Fit the model
model.fit(x_train, y_train, validation_split=0.2, epochs=50, batch_size=1000, verbose=1)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=1)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Train on 40000 samples, validate on 10000 samples
Epoch 1/50
40000/40000 [==============================] - 47s - loss: 2.6841 - acc: 0.1287 - val_loss: 2.4800 - val_acc: 0.1458
Epoch 2/50
25000/40000 [=================>............] - ETA: 15s - loss: 2.4119 - acc: 0.1698

KeyboardInterrupt: 

GridSearch for optimizer

In [45]:
num_pixels = 32 * 32 * 3
num_classes = 10

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
model = KerasClassifier(build_fn=baseline_model, epochs=50, batch_size=1000, verbose=1)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(x_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
33333/33333 [==============================] - 35s - loss: 7.7897 - acc: 0.1353    
Epoch 2/50
33333/33333 [==============================] - 35s - loss: 5.8730 - acc: 0.1964    
Epoch 3/50
33333/33333 [==============================] - 35s - loss: 5.3171 - acc: 0.2147    
Epoch 4/50
33333/33333 [==============================] - 36s - loss: 3.7064 - acc: 0.2428    
Epoch 5/50
33333/33333 [==============================] - 36s - loss: 2.2173 - acc: 0.2814    
Epoch 6/50
33333/33333 [==============================] - 39s - loss: 2.0246 - acc: 0.3165    
Epoch 7/50
33333/33333 [==============================] - 37s - loss: 1.9810 - acc: 0.3236    
Epoch 8/50
33333/33333 [==============================] - 38s - loss: 1.8751 - acc: 0.3479    
Epoch 9/50
33333/33333 [==============================] - 39s - loss: 1.8154 - acc: 0.3633    
Epoch 10/50
33333/33333 [==============================] - 39s - loss: 1.7906 - acc: 0.3737    
Epoch 11/50
33333/33333 [========================

33333/33333 [==============================] - 34s - loss: 1.5444 - acc: 0.4654    
Epoch 36/50
33333/33333 [==============================] - 34s - loss: 1.5467 - acc: 0.4659    
Epoch 37/50
33333/33333 [==============================] - 34s - loss: 1.5379 - acc: 0.4704    
Epoch 38/50
33333/33333 [==============================] - 34s - loss: 1.5330 - acc: 0.4720    
Epoch 39/50
33333/33333 [==============================] - 34s - loss: 1.5294 - acc: 0.4727    
Epoch 40/50
33333/33333 [==============================] - 34s - loss: 1.5218 - acc: 0.4727    
Epoch 41/50
33333/33333 [==============================] - 34s - loss: 1.5208 - acc: 0.4770    
Epoch 42/50
33333/33333 [==============================] - 34s - loss: 1.5195 - acc: 0.4765    
Epoch 43/50
33333/33333 [==============================] - 34s - loss: 1.5116 - acc: 0.4787    
Epoch 44/50
33333/33333 [==============================] - 34s - loss: 1.5058 - acc: 0.4822    
Epoch 45/50
33333/33333 [===========================

33333/33333 [==============================] - 35s - loss: 14.5239 - acc: 0.0989    
Epoch 19/50
33333/33333 [==============================] - 35s - loss: 14.5239 - acc: 0.0989    
Epoch 20/50
33333/33333 [==============================] - 35s - loss: 14.5239 - acc: 0.0989    
Epoch 21/50
33333/33333 [==============================] - 36s - loss: 14.5239 - acc: 0.0989    
Epoch 22/50
33333/33333 [==============================] - 36s - loss: 14.5239 - acc: 0.0989    
Epoch 23/50
33333/33333 [==============================] - 36s - loss: 14.5239 - acc: 0.0989    
Epoch 24/50
33333/33333 [==============================] - 36s - loss: 14.5239 - acc: 0.0989    
Epoch 25/50
33333/33333 [==============================] - 36s - loss: 14.5239 - acc: 0.0989    
Epoch 26/50
33333/33333 [==============================] - 36s - loss: 14.5239 - acc: 0.0989    
Epoch 27/50
33333/33333 [==============================] - 36s - loss: 14.5239 - acc: 0.0989    
Epoch 28/50
33333/33333 [=================

33333/33333 [==============================] - 16s    
Epoch 1/50
33334/33334 [==============================] - 36s - loss: 14.0921 - acc: 0.1012    
Epoch 2/50
33334/33334 [==============================] - 36s - loss: 14.4702 - acc: 0.1010    
Epoch 3/50
33334/33334 [==============================] - 36s - loss: 14.4915 - acc: 0.1009    
Epoch 4/50
33334/33334 [==============================] - 36s - loss: 14.4915 - acc: 0.1009    
Epoch 5/50
33334/33334 [==============================] - 36s - loss: 14.4915 - acc: 0.1009    
Epoch 6/50
33334/33334 [==============================] - 36s - loss: 14.4915 - acc: 0.1009    
Epoch 7/50
33334/33334 [==============================] - 36s - loss: 14.4915 - acc: 0.1009    
Epoch 8/50
33334/33334 [==============================] - 36s - loss: 14.4915 - acc: 0.1009    
Epoch 9/50
33334/33334 [==============================] - 36s - loss: 14.4915 - acc: 0.1009    
Epoch 10/50
33334/33334 [==============================] - 36s - loss: 14.4915 - 

33333/33333 [==============================] - 36s - loss: 14.5161 - acc: 0.0994    
Epoch 34/50
33333/33333 [==============================] - 36s - loss: 14.5161 - acc: 0.0994    
Epoch 35/50
33333/33333 [==============================] - 40s - loss: 14.5161 - acc: 0.0994    
Epoch 36/50
33333/33333 [==============================] - 36s - loss: 14.5161 - acc: 0.0994    
Epoch 37/50
33333/33333 [==============================] - 36s - loss: 14.5161 - acc: 0.0994    
Epoch 38/50
33333/33333 [==============================] - 37s - loss: 14.5161 - acc: 0.0994    
Epoch 39/50
33333/33333 [==============================] - 36s - loss: 14.5161 - acc: 0.0994    
Epoch 40/50
33333/33333 [==============================] - 36s - loss: 14.5161 - acc: 0.0994    
Epoch 41/50
33333/33333 [==============================] - 35s - loss: 14.5161 - acc: 0.0994    
Epoch 42/50
33333/33333 [==============================] - 35s - loss: 14.5161 - acc: 0.0994    
Epoch 43/50
33333/33333 [=================

...

33334/33334 [==============================] - 36s - loss: 7.1539 - acc: 0.3365    
Epoch 48/50
33334/33334 [==============================] - 36s - loss: 7.1504 - acc: 0.3403    
Epoch 49/50
33334/33334 [==============================] - 36s - loss: 7.2485 - acc: 0.3172    
Epoch 50/50
33334/33334 [==============================] - 16s    
Epoch 1/50
33333/33333 [==============================] - 36s - loss: 13.5112 - acc: 0.1040    
Epoch 2/50
33333/33333 [==============================] - 36s - loss: 13.2285 - acc: 0.1268    
Epoch 3/50
33333/33333 [==============================] - 36s - loss: 11.9979 - acc: 0.1679    
Epoch 4/50
33333/33333 [==============================] - 36s - loss: 11.6602 - acc: 0.1955    
Epoch 5/50
33333/33333 [==============================] - 36s - loss: 11.5656 - acc: 0.2079    
Epoch 6/50
33333/33333 [==============================] - 36s - loss: 11.5544 - acc: 0.2066    
Epoch 7/50
33333/33333 [==============================] - 36s - loss: 11.5389 - a

33333/33333 [==============================] - 36s - loss: 10.2160 - acc: 0.2253    
Epoch 30/50
33333/33333 [==============================] - 36s - loss: 10.0882 - acc: 0.2530    
Epoch 31/50
33333/33333 [==============================] - 36s - loss: 10.0423 - acc: 0.2594    
Epoch 32/50
33333/33333 [==============================] - 36s - loss: 10.1394 - acc: 0.2404    
Epoch 33/50
33333/33333 [==============================] - 36s - loss: 10.0226 - acc: 0.2625    
Epoch 34/50
33333/33333 [==============================] - 36s - loss: 10.0591 - acc: 0.2567    
Epoch 35/50
33333/33333 [==============================] - 36s - loss: 10.1814 - acc: 0.2326    
Epoch 36/50
33333/33333 [==============================] - 36s - loss: 10.1798 - acc: 0.2365    
Epoch 37/50
33333/33333 [==============================] - 36s - loss: 10.1123 - acc: 0.2522    
Epoch 38/50
33333/33333 [==============================] - 36s - loss: 10.0555 - acc: 0.2559    
Epoch 39/50
33333/33333 [=================

33333/33333 [==============================] - 37s - loss: 14.5239 - acc: 0.0989    
Epoch 12/50
33333/33333 [==============================] - 37s - loss: 14.5239 - acc: 0.0989    
Epoch 13/50
33333/33333 [==============================] - 37s - loss: 14.5239 - acc: 0.0989    
Epoch 14/50
33333/33333 [==============================] - 37s - loss: 14.5239 - acc: 0.0989    
Epoch 15/50
33333/33333 [==============================] - 37s - loss: 14.5239 - acc: 0.0989    
Epoch 16/50
33333/33333 [==============================] - 37s - loss: 14.5239 - acc: 0.0989    
Epoch 17/50
33333/33333 [==============================] - 37s - loss: 14.5239 - acc: 0.0989    
Epoch 18/50
33333/33333 [==============================] - 37s - loss: 14.5239 - acc: 0.0989    
Epoch 19/50
33333/33333 [==============================] - 37s - loss: 14.5239 - acc: 0.0989    
Epoch 20/50
33333/33333 [==============================] - 37s - loss: 14.5239 - acc: 0.0989    
Epoch 21/50
33333/33333 [=================

33333/33333 [==============================] - 37s - loss: 14.4944 - acc: 0.1007    
Epoch 45/50
33333/33333 [==============================] - 37s - loss: 14.4944 - acc: 0.1007    
Epoch 46/50
33333/33333 [==============================] - 37s - loss: 14.4944 - acc: 0.1007    
Epoch 47/50
33333/33333 [==============================] - 37s - loss: 14.4944 - acc: 0.1007    
Epoch 48/50
33333/33333 [==============================] - 37s - loss: 14.4944 - acc: 0.1007    
Epoch 49/50
33333/33333 [==============================] - 37s - loss: 14.4944 - acc: 0.1007    
Epoch 50/50
33333/33333 [==============================] - 16s    
Epoch 1/50
33334/33334 [==============================] - 38s - loss: 13.9469 - acc: 0.1012    
Epoch 2/50
33334/33334 [==============================] - 37s - loss: 14.4939 - acc: 0.1008    
Epoch 3/50
33334/33334 [==============================] - 37s - loss: 14.4939 - acc: 0.1008    
Epoch 4/50
33334/33334 [==============================] - 37s - loss: 14.49

50000/50000 [==============================] - 56s - loss: 1.3692 - acc: 0.5204    
Epoch 27/50
50000/50000 [==============================] - 56s - loss: 1.3578 - acc: 0.5301    
Epoch 28/50
50000/50000 [==============================] - 56s - loss: 1.3343 - acc: 0.5373    
Epoch 29/50
50000/50000 [==============================] - 56s - loss: 1.3246 - acc: 0.5394    
Epoch 30/50
50000/50000 [==============================] - 56s - loss: 1.3192 - acc: 0.5420    
Epoch 31/50
50000/50000 [==============================] - 56s - loss: 1.3101 - acc: 0.5470    
Epoch 32/50
50000/50000 [==============================] - 55s - loss: 1.2953 - acc: 0.5510    
Epoch 33/50
50000/50000 [==============================] - 56s - loss: 1.2841 - acc: 0.5524    
Epoch 34/50
50000/50000 [==============================] - 56s - loss: 1.2684 - acc: 0.5638    
Epoch 35/50
50000/50000 [==============================] - 56s - loss: 1.2559 - acc: 0.5648    
Epoch 36/50
50000/50000 [===========================

GridSearch for batch size and epochs

In [46]:
num_pixels = 32 * 32 * 3
num_classes = 10

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
model = KerasClassifier(build_fn=baseline_model, verbose=2)

# define the grid search parameters
batch_size = [500, 1000, 2500]
epochs = [100, 250, 500]
learning_rate = .001 #[.0001, .001, .01, .1, 1]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/100
36s - loss: 6.1566 - acc: 0.1706
Epoch 2/100
35s - loss: 3.2880 - acc: 0.2586
Epoch 3/100
35s - loss: 2.0577 - acc: 0.3088
Epoch 4/100
35s - loss: 1.9042 - acc: 0.3453
Epoch 5/100
35s - loss: 1.8328 - acc: 0.3596
Epoch 6/100
35s - loss: 1.7646 - acc: 0.3822
Epoch 7/100
35s - loss: 1.7454 - acc: 0.3901
Epoch 8/100
35s - loss: 1.6891 - acc: 0.4079
Epoch 9/100
35s - loss: 1.6827 - acc: 0.4106
Epoch 10/100
35s - loss: 1.6788 - acc: 0.4126
Epoch 11/100
35s - loss: 1.6482 - acc: 0.4231
Epoch 12/100
35s - loss: 1.6293 - acc: 0.4309
Epoch 13/100
36s - loss: 1.6129 - acc: 0.4376
Epoch 14/100
35s - loss: 1.5910 - acc: 0.4427
Epoch 15/100
35s - loss: 1.5914 - acc: 0.4468
Epoch 16/100
35s - loss: 1.5754 - acc: 0.4530
Epoch 17/100
35s - loss: 1.5640 - acc: 0.4539
Epoch 18/100
35s - loss: 1.5552 - acc: 0.4606
Epoch 19/100
35s - loss: 1.5473 - acc: 0.4603
Epoch 20/100
35s - loss: 1.5352 - acc: 0.4690
Epoch 21/100
35s - loss: 1.5288 - acc: 0.4703
Epoch 22/100
35s - loss: 1.5219 - acc: 0.47

35s - loss: 1.2603 - acc: 0.5739
Epoch 81/100
35s - loss: 1.2515 - acc: 0.5797
Epoch 82/100
35s - loss: 1.2466 - acc: 0.5820
Epoch 83/100
35s - loss: 1.2484 - acc: 0.5798
Epoch 84/100
35s - loss: 1.2447 - acc: 0.5829
Epoch 85/100
35s - loss: 1.2422 - acc: 0.5825
Epoch 86/100
35s - loss: 1.2375 - acc: 0.5839
Epoch 87/100
35s - loss: 1.2344 - acc: 0.5852
Epoch 88/100
35s - loss: 1.2360 - acc: 0.5848
Epoch 89/100
35s - loss: 1.2296 - acc: 0.5861
Epoch 90/100
35s - loss: 1.2248 - acc: 0.5877
Epoch 91/100
35s - loss: 1.2237 - acc: 0.5918
Epoch 92/100
35s - loss: 1.2192 - acc: 0.5917
Epoch 93/100
35s - loss: 1.2190 - acc: 0.5927
Epoch 94/100
35s - loss: 1.2145 - acc: 0.5924
Epoch 95/100
35s - loss: 1.2096 - acc: 0.5944
Epoch 96/100
35s - loss: 1.2086 - acc: 0.5965
Epoch 97/100
35s - loss: 1.2057 - acc: 0.5980
Epoch 98/100
35s - loss: 1.1999 - acc: 0.6025
Epoch 99/100
35s - loss: 1.1988 - acc: 0.6008
Epoch 100/100
35s - loss: 1.1957 - acc: 0.6015
Epoch 1/100
35s - loss: 6.5842 - acc: 0.1598
E

35s - loss: 1.3359 - acc: 0.5427
Epoch 60/250
35s - loss: 1.3297 - acc: 0.5466
Epoch 61/250
35s - loss: 1.3239 - acc: 0.5508
Epoch 62/250
35s - loss: 1.3179 - acc: 0.5537
Epoch 63/250
35s - loss: 1.3135 - acc: 0.5534
Epoch 64/250
35s - loss: 1.3086 - acc: 0.5564
Epoch 65/250
35s - loss: 1.3061 - acc: 0.5570
Epoch 66/250
35s - loss: 1.3023 - acc: 0.5589
Epoch 67/250
35s - loss: 1.3035 - acc: 0.5564
Epoch 68/250
35s - loss: 1.2973 - acc: 0.5602
Epoch 69/250
35s - loss: 1.2939 - acc: 0.5629
Epoch 70/250
36s - loss: 1.2898 - acc: 0.5626
Epoch 71/250
35s - loss: 1.2892 - acc: 0.5641
Epoch 72/250
35s - loss: 1.2812 - acc: 0.5650
Epoch 73/250
35s - loss: 1.2769 - acc: 0.5698
Epoch 74/250
35s - loss: 1.2729 - acc: 0.5701
Epoch 75/250
35s - loss: 1.2716 - acc: 0.5699
Epoch 76/250
35s - loss: 1.2707 - acc: 0.5703
Epoch 77/250
35s - loss: 1.2649 - acc: 0.5727
Epoch 78/250
35s - loss: 1.2624 - acc: 0.5738
Epoch 79/250
35s - loss: 1.2606 - acc: 0.5750
Epoch 80/250
35s - loss: 1.2571 - acc: 0.5748
E

35s - loss: 0.9057 - acc: 0.7133
Epoch 236/250
35s - loss: 0.9002 - acc: 0.7131
Epoch 237/250
35s - loss: 0.8983 - acc: 0.7173
Epoch 238/250
35s - loss: 0.9019 - acc: 0.7146
Epoch 239/250
35s - loss: 0.8968 - acc: 0.7150
Epoch 240/250
35s - loss: 0.8985 - acc: 0.7123
Epoch 241/250
35s - loss: 0.8899 - acc: 0.7187
Epoch 242/250
35s - loss: 0.8867 - acc: 0.7200
Epoch 243/250
35s - loss: 0.8892 - acc: 0.7199
Epoch 244/250
35s - loss: 0.8890 - acc: 0.7169
Epoch 245/250
35s - loss: 0.8850 - acc: 0.7195
Epoch 246/250
35s - loss: 0.8876 - acc: 0.7188
Epoch 247/250
35s - loss: 0.8807 - acc: 0.7226
Epoch 248/250
35s - loss: 0.8783 - acc: 0.7226
Epoch 249/250
35s - loss: 0.8878 - acc: 0.7151
Epoch 250/250
35s - loss: 0.8820 - acc: 0.7214
Epoch 1/250
36s - loss: 7.1498 - acc: 0.1541
Epoch 2/250
35s - loss: 3.5037 - acc: 0.2486
Epoch 3/250
35s - loss: 2.0553 - acc: 0.3082
Epoch 4/250
35s - loss: 1.9140 - acc: 0.3384
Epoch 5/250
35s - loss: 1.8421 - acc: 0.3566
Epoch 6/250
35s - loss: 1.7754 - acc:

39s - loss: 1.0484 - acc: 0.6551
Epoch 163/250
40s - loss: 1.0435 - acc: 0.6608
Epoch 164/250
39s - loss: 1.0382 - acc: 0.6603
Epoch 165/250
43s - loss: 1.0353 - acc: 0.6626
Epoch 166/250
40s - loss: 1.0319 - acc: 0.6632
Epoch 167/250
40s - loss: 1.0359 - acc: 0.6623
Epoch 168/250
40s - loss: 1.0308 - acc: 0.6661
Epoch 169/250
40s - loss: 1.0224 - acc: 0.6693
Epoch 170/250
40s - loss: 1.0228 - acc: 0.6689
Epoch 171/250
41s - loss: 1.0238 - acc: 0.6669
Epoch 172/250
42s - loss: 1.0219 - acc: 0.6673
Epoch 173/250
41s - loss: 1.0262 - acc: 0.6653
Epoch 174/250
40s - loss: 1.0168 - acc: 0.6678
Epoch 175/250
41s - loss: 1.0103 - acc: 0.6718
Epoch 176/250
42s - loss: 1.0147 - acc: 0.6683
Epoch 177/250
42s - loss: 1.0123 - acc: 0.6725
Epoch 178/250
42s - loss: 1.0047 - acc: 0.6748
Epoch 179/250
42s - loss: 1.0059 - acc: 0.6737
Epoch 180/250
42s - loss: 1.0100 - acc: 0.6740
Epoch 181/250
42s - loss: 1.0039 - acc: 0.6759
Epoch 182/250
42s - loss: 1.0085 - acc: 0.6725
Epoch 183/250
42s - loss: 1

...

36s - loss: 1.3258 - acc: 0.5516
Epoch 258/500
32s - loss: 1.3214 - acc: 0.5540
Epoch 259/500
32s - loss: 1.3214 - acc: 0.5533
Epoch 260/500
32s - loss: 1.3193 - acc: 0.5555
Epoch 261/500
33s - loss: 1.3253 - acc: 0.5493
Epoch 262/500
33s - loss: 1.3274 - acc: 0.5510
Epoch 263/500
33s - loss: 1.3193 - acc: 0.5559
Epoch 264/500
33s - loss: 1.3214 - acc: 0.5542
Epoch 265/500
33s - loss: 1.3158 - acc: 0.5567
Epoch 266/500
33s - loss: 1.3151 - acc: 0.5567
Epoch 267/500
33s - loss: 1.3185 - acc: 0.5538
Epoch 268/500
33s - loss: 1.3128 - acc: 0.5569
Epoch 269/500
33s - loss: 1.3188 - acc: 0.5533
Epoch 270/500
33s - loss: 1.3189 - acc: 0.5543
Epoch 271/500
33s - loss: 1.3163 - acc: 0.5546
Epoch 272/500
33s - loss: 1.3142 - acc: 0.5554
Epoch 273/500
33s - loss: 1.3194 - acc: 0.5538
Epoch 274/500
33s - loss: 1.3139 - acc: 0.5549
Epoch 275/500
33s - loss: 1.3073 - acc: 0.5588
Epoch 276/500
33s - loss: 1.3049 - acc: 0.5617
Epoch 277/500
33s - loss: 1.3084 - acc: 0.5597
Epoch 278/500
33s - loss: 1

33s - loss: 1.2053 - acc: 0.5960
Epoch 433/500
37s - loss: 1.2008 - acc: 0.6016
Epoch 434/500
34s - loss: 1.2036 - acc: 0.5990
Epoch 435/500
33s - loss: 1.2056 - acc: 0.5968
Epoch 436/500
32s - loss: 1.1985 - acc: 0.6027
Epoch 437/500
32s - loss: 1.1987 - acc: 0.6019
Epoch 438/500
32s - loss: 1.2027 - acc: 0.5989
Epoch 439/500
33s - loss: 1.2025 - acc: 0.5994
Epoch 440/500
33s - loss: 1.1916 - acc: 0.6055
Epoch 441/500
33s - loss: 1.2011 - acc: 0.5991
Epoch 442/500
33s - loss: 1.1994 - acc: 0.5994
Epoch 443/500
33s - loss: 1.1978 - acc: 0.6006
Epoch 444/500
33s - loss: 1.2017 - acc: 0.5981
Epoch 445/500
33s - loss: 1.1990 - acc: 0.5986
Epoch 446/500
33s - loss: 1.1937 - acc: 0.6021
Epoch 447/500
33s - loss: 1.1915 - acc: 0.6038
Epoch 448/500
33s - loss: 1.1893 - acc: 0.6056
Epoch 449/500
33s - loss: 1.1943 - acc: 0.5994
Epoch 450/500
33s - loss: 1.1923 - acc: 0.6042
Epoch 451/500
33s - loss: 1.1985 - acc: 0.5979
Epoch 452/500
33s - loss: 1.1888 - acc: 0.6052
Epoch 453/500
33s - loss: 1

57s - loss: 1.1360 - acc: 0.6196
Epoch 110/500
57s - loss: 1.1360 - acc: 0.6187
Epoch 111/500
57s - loss: 1.1282 - acc: 0.6211
Epoch 112/500
58s - loss: 1.1310 - acc: 0.6232
Epoch 113/500
57s - loss: 1.1247 - acc: 0.6240
Epoch 114/500
57s - loss: 1.1242 - acc: 0.6242
Epoch 115/500
57s - loss: 1.1211 - acc: 0.6275
Epoch 116/500
57s - loss: 1.1201 - acc: 0.6241
Epoch 117/500
57s - loss: 1.1120 - acc: 0.6286
Epoch 118/500
57s - loss: 1.1114 - acc: 0.6295
Epoch 119/500
57s - loss: 1.1096 - acc: 0.6311
Epoch 120/500
57s - loss: 1.1098 - acc: 0.6300
Epoch 121/500
57s - loss: 1.1084 - acc: 0.6303
Epoch 122/500
57s - loss: 1.1026 - acc: 0.6324
Epoch 123/500
57s - loss: 1.0998 - acc: 0.6345
Epoch 124/500
57s - loss: 1.0954 - acc: 0.6351
Epoch 125/500
57s - loss: 1.0957 - acc: 0.6356
Epoch 126/500
57s - loss: 1.0890 - acc: 0.6379
Epoch 127/500
57s - loss: 1.0935 - acc: 0.6359
Epoch 128/500
57s - loss: 1.0899 - acc: 0.6363
Epoch 129/500
57s - loss: 1.0873 - acc: 0.6386
Epoch 130/500
63s - loss: 1

54s - loss: 0.7926 - acc: 0.7491
Epoch 285/500
54s - loss: 0.7978 - acc: 0.7454
Epoch 286/500
54s - loss: 0.7886 - acc: 0.7506
Epoch 287/500
55s - loss: 0.7935 - acc: 0.7484
Epoch 288/500
54s - loss: 0.7908 - acc: 0.7483
Epoch 289/500
54s - loss: 0.7809 - acc: 0.7531
Epoch 290/500
54s - loss: 0.7870 - acc: 0.7504
Epoch 291/500
54s - loss: 0.7830 - acc: 0.7524
Epoch 292/500
54s - loss: 0.7809 - acc: 0.7516
Epoch 293/500
54s - loss: 0.7832 - acc: 0.7507
Epoch 294/500
54s - loss: 0.7750 - acc: 0.7553
Epoch 295/500
54s - loss: 0.7725 - acc: 0.7580
Epoch 296/500
54s - loss: 0.7756 - acc: 0.7511
Epoch 297/500
54s - loss: 0.7725 - acc: 0.7565
Epoch 298/500
54s - loss: 0.7787 - acc: 0.7521
Epoch 299/500
54s - loss: 0.7745 - acc: 0.7549
Epoch 300/500
54s - loss: 0.7712 - acc: 0.7557
Epoch 301/500
54s - loss: 0.7708 - acc: 0.7563
Epoch 302/500
54s - loss: 0.7627 - acc: 0.7594
Epoch 303/500
54s - loss: 0.7557 - acc: 0.7631
Epoch 304/500
54s - loss: 0.7678 - acc: 0.7556
Epoch 305/500
54s - loss: 0

54s - loss: 0.5549 - acc: 0.8386
Epoch 460/500
54s - loss: 0.5682 - acc: 0.8293
Epoch 461/500
54s - loss: 0.5556 - acc: 0.8375
Epoch 462/500
55s - loss: 0.5535 - acc: 0.8392
Epoch 463/500
54s - loss: 0.5619 - acc: 0.8351
Epoch 464/500
54s - loss: 0.5541 - acc: 0.8378
Epoch 465/500
53s - loss: 0.5627 - acc: 0.8322
Epoch 466/500
54s - loss: 0.5585 - acc: 0.8348
Epoch 467/500
54s - loss: 0.5456 - acc: 0.8406
Epoch 468/500
54s - loss: 0.5492 - acc: 0.8384
Epoch 469/500
58s - loss: 0.5499 - acc: 0.8378
Epoch 470/500
59s - loss: 0.5464 - acc: 0.8407
Epoch 471/500
59s - loss: 0.5487 - acc: 0.8407
Epoch 472/500
61s - loss: 0.5554 - acc: 0.8382
Epoch 473/500
57s - loss: 0.5508 - acc: 0.8390
Epoch 474/500
57s - loss: 0.5412 - acc: 0.8449
Epoch 475/500
60s - loss: 0.5392 - acc: 0.8455
Epoch 476/500
54s - loss: 0.5423 - acc: 0.8404
Epoch 477/500
53s - loss: 0.5371 - acc: 0.8459
Epoch 478/500
54s - loss: 0.5289 - acc: 0.8503
Epoch 479/500
54s - loss: 0.5489 - acc: 0.8366
Epoch 480/500
54s - loss: 0

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
# from keras import backend as K
# K.set_image_dim_ordering('tf')

In [ ]:
# # load data
# (X_train, y_train), (X_test, y_test) = cifar10.load_data()
# # reshape to be [samples][pixels][width][height]
# X_train = X_train.reshape(X_train.shape[0], 3, 32, 32).astype('float32')
# X_test = X_test.reshape(X_test.shape[0], 3, 32, 32).astype('float32')

In [ ]:
# # normalize inputs from 0-255 to 0-1
# x_train = x_train / 255
# x_test = x_test / 255
# # one hot encode outputs
# y_train = np_utils.to_categorical(y_train)
# y_test = np_utils.to_categorical(y_test)
# num_classes = y_train.shape[1]
# num_classes

In [ ]:
X_train = x_train.reshape(50000, 32, 32, 3)
X_test = x_test.reshape(10000, 32, 32, 3)

In [ ]:
def convolution_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(32, 32, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    # Compile model
    adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [ ]:
# build the model
model = convolution_model()
# Fit the model
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=200, verbose=1)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=1)
print("Convolution Error: %.2f%%" % (100-scores[1]*100))

In [ ]:
def convolution2_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(32, 32, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    # Compile model
    adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

# build the model
model = convolution2_model()
# Fit the model
model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=200, verbose=1)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=1)
print("Convolution Error with 2 hidden layers: %.2f%%" % (100-scores[1]*100))

In [ ]:
def convolution2b_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    # Compile model
    adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

# build the model
model = convolution2b_model()
# Fit the model
model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=100, verbose=1)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=1)
print("Convolution Error with 2 hidden layers: %.2f%%" % (100-scores[1]*100))

In [ ]:
def convolution2b_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    # Compile model
    adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

# build the model
model = convolution2b_model()
# Fit the model
results = model.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=1000, verbose=1)
plt.plot(results.history['loss'])
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=1)
print("Convolution Error with 2 hidden layers: %.2f%%" % (100-scores[1]*100))

In [ ]:
plt.plot(results.history['loss'])
plt.plot(results.history['acc'])
plt.plot(results.history['val_loss'])
plt.plot(results.history['val_acc'])

In [ ]:
scores

https://cambridgespark.com/content/tutorials/convolutional-neural-networks-with-keras/index.html

In [ ]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

In [ ]:
def convolution2c_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (1, 1), input_shape=(32, 32, 3), activation='relu'))
    model.add(Conv2D(32, (1, 1), input_shape=(32, 32, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    # Compile model
    adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

# build the model
model = convolution2c_model()
# Fit the model
results = model.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=1000, verbose=1)
plt.plot(results.history['loss'])
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=1)
print("Convolution Error with 2 hidden layers: %.2f%%" % (100-scores[1]*100))

In [ ]:
def convolution3_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    # Compile model
    adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

# build the model
model = convolution3_model()
# Fit the model
results = model.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=1000, verbose=1)
plt.plot(results.history['loss'])
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=1)
scores
print("Convolution Error with 2 hidden layers: %.2f%%" % (100-scores[1]*100))

In [ ]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def convolution3_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    # Compile model
    adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model
# # fix random seed for reproducibility
# seed = 7
# numpy.random.seed(seed)
# # load dataset
# dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# # split into input (X) and output (Y) variables
# X = dataset[:,0:8]
# Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=convolution3_model, verbose=1)
# define the grid search parameters
batch_size = [10, 50, 100, 250, 500, 1000]
epochs = [20, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

The following code was adapted from:
https://notebooks.azure.com/anon-yqopra/libraries/pydata/html/workspace.ipynb

In [ ]:
len(x_train), len(y_train), len(x_test), len(y_test)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

def display_images_and_labels(images, labels):
    """Display the first image of each label."""
    unique_labels = set(labels)
    plt.figure(figsize=(15, 15))
    i = 1
    for label in unique_labels:
        # Pick the first image for each label.
        image = images[labels.index(label)]
        plt.subplot(8, 8, i)  # A grid of 8 rows x 8 columns
        plt.axis('off')
        plt.title("Label {0} ({1})".format(label, labels.count(label)))
        i += 1
        _ = plt.imshow(image)
    plt.show()

In [ ]:
display_images_and_labels(x_train, y_train)